# Import modules

In [1]:
import ROOT
%jsroot on
#ROOT.gROOT.ProcessLine("gErrorIgnoreLevel = 2001;")
#ROOT.gROOT.LoadMacro('RooCMSShape.cc+')
#ROOT.gROOT.LoadMacro('RooErfExp.cc+')

Welcome to JupyROOT 6.26/02


# Read root file

In [2]:
year = 2018
f = ROOT.TFile("DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/DYTreeNoBCut2018Skim.root", "read")
#hNoCorr = f.Get("demo/hZNoRoccor")
#hCorr = f.Get("demo/hZRoccor")
zhTree = f.Get("dyTree")

# Fitting PDF

In [3]:
mass = ROOT.RooRealVar("mass", "mass of Z boson", 91, 90, 92)
gamma = ROOT.RooRealVar("gamma", "width of BW", 3.65555, 0, 5)
sigma = ROOT.RooRealVar("sigma", "width of Gauss", 7.31860e-01, 0, 5)
massCorr = ROOT.RooRealVar("massCorr", "mass of Z boson", 91, 90, 92)
gammaCorr = ROOT.RooRealVar("gammaCorr", "width of BW", 3.7, 1, 5)
sigmaCorr = ROOT.RooRealVar("sigmaCorr", "width of Gauss", 7.77316e-01, 0, 2)
fitNoCorrPdf = ROOT.RooVoigtian("fitPdf", "voigtian", x, mass, gamma, sigma)
fitCorrPdf = ROOT.RooVoigtian("fitPdf", "voigtian", x, massCorr, gammaCorr, sigmaCorr)

In [3]:
zM = ROOT.RooRealVar("z_mass_no_roccor", "z_mass_no_roccor", 75, 105)
ds = ROOT.RooDataSet("ds", "ds", zhTree, ROOT.RooArgSet(zM))
kest = ROOT.RooKeysPdf("kest", "kest", zM, ds, ROOT.RooKeysPdf.MirrorBoth)

In [5]:
nEvents = zhTree.GetEntries()

mass = ROOT.RooRealVar("mass", "mass of Z boson", 0, -5, 5)
sigma = ROOT.RooRealVar("sigma", "width of Gauss", 0.9, 0.05, 5.0)
gauss = ROOT.RooGaussian("gauss", "gauss", zM, mass, sigma)
sig = ROOT.RooFFTConvPdf("sig", "sig", zM, kest, gauss)

acms = ROOT.RooRealVar("acms", "acms", 70., 50., 90.)
beta = ROOT.RooRealVar("beta", "beta", 0.05, 0.01, 0.08)
gamma = ROOT.RooRealVar("gamma", "gamma", 0.1, -2, 2)
peak = ROOT.RooRealVar("peak", "peak", 91, 90, 92)

#bkg = ROOT.RooCMSShape("bkg", "bkg", zM, acms, beta, gamma, peak)

#fracSig = ROOT.RooRealVar("fracSig", "fracSig", 0.9 * nEvents, 0.5, 1.1 * nEvents)
#fracBkg = ROOT.RooRealVar("fracBkg", "fracBkg", 0.9 * nEvents, 0.5, 1.1 * nEvents)
#model = ROOT.RooAddPdf("model", "model", ROOT.RooArgList(sig, bkg), ROOT.RooArgList(fracSig, fracBkg))

# Fit

In [6]:
sig.fitTo(ds, ROOT.RooFit.NumCPU(12, 0))

[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Eval -- RooRealVar::setRange(z_mass_no_roccor) new range named 'refrange_fft_sig' created with bounds [75,105]
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600003b605a0 with pdf kest_CONV_gauss_CACHE_Obs[z_mass_no_roccor]_NORM_z_mass_no_roccor for nset (z_mass_no_roccor) with code 0
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 12 remote server process.
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mass         0.00000e+00  1.00000e+00   -5.00000e+00  5.00000e+00
     2 sigma        9.00000e-01  4.25000e-01    5.00000e-02  5.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NO

# Plot

In [7]:
xFrameCorr = zM.frame()
ds.plotOn(xFrameCorr)
sig.plotOn(xFrameCorr)
c = ROOT.TCanvas()
c.Draw()
xFrameCorr.Draw()

[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600003b6eb20 with pdf kest_CONV_gauss_CACHE_Obs[z_mass_no_roccor]_NORM_z_mass_no_roccor for nset (z_mass_no_roccor) with code 0


In [8]:
print(xFrameCorr.chiSquare(2))

0.783456116553001


In [9]:
cPull = ROOT.TCanvas()
pullHist = xFrameCorr.pullHist()
cPull.Draw()
pullHist.Draw("ap")

# Create Hist

In [10]:
hPdf = sig.createHistogram("hPdf", zM, ROOT.RooFit.Binning(100, 75, 105))
cHist = ROOT.TCanvas()
cHist.Draw()
hPdf.Draw()
print(hPdf.GetMean())
print(hPdf.GetMeanError())

90.47605903177603
4.026022242230577
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600003b62a30 with pdf kest_CONV_gauss_CACHE_Obs[z_mass_no_roccor]_NORM_z_mass_no_roccor for nset (z_mass_no_roccor) with code 0 from preexisting content.


In [54]:
sig.Print("t")

0x115047000 RooFFTConvPdf::sig = [#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600000dfc000 with pdf kest_CONV_gauss_CACHE_Obs[z_mass] for nset () with code 0 from preexisting content.
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600000dfc1e0 with pdf kest_CONV_gauss_CACHE_Obs[z_mass]_NORM_z_mass for nset (z_mass) with code 1 from preexisting content.
0.138655 [Auto,Dirty] 
--- RooAbsCachedPdf begin cache ---
[0] Configuration for observables ()
[0]0x10503c800 RooHistPdf::kest_CONV_gauss_CACHE_Obs[z_mass] = 136443 [Auto,Dirty] 
[0]  0x156a386f0/V- RooRealVar::z_mass = 90
[0]  0x11482f5b0/V- RooRealVar::mass = 0.00114329 +/- 0.00452108
[0]  0x11482f930/V- RooRealVar::sigma = 0.05 +/- 0.00785207
[1] Configuration for observables (z_mass)
[1]0x105058000 RooHistPdf::kest_CONV_gauss_CACHE_Obs[z_mass]_NORM_z_mass = 136443/984050 [Auto,Clean] 
[1]  0x156a386f0/V- RooRealVar::z_mass = 90
[1]  0x11482f5b0/V- RooRealVar::mass = 0.00114329 +/

In [100]:
cComp = ROOT.TCanvas("cComp", "cComp", 800, 600)
topPad = ROOT.TPad("topPad", "topPad", 0, 0.3, 1, 1)
bottomPad = ROOT.TPad("bottomPad", "bottomPad", 0, 0, 1, 0.3)
cComp.Draw()
topPad.Draw()
bottomPad.Draw()
topPad.cd()
topPad.SetBottomMargin(0)
yearMark = ROOT.TLatex()
yearMark.SetTextFont(42)
xFrameCorr.SetTitle("")
xFrameCorr.Draw()
xFrameCorr.GetYaxis().SetTitle("A.U. / 0.3 GeV")
xFrameCorr.GetYaxis().SetTitleSize(0.05)
xFrameCorr.GetYaxis().SetLabelSize(0.05)
xFrameCorr.GetYaxis().SetTitleOffset(1)
xFrameCorr.GetXaxis().SetLimits(74.5, 105.5)
yearMark.DrawLatexNDC(0.6, 0.92, "Before RoccoR (13 TeV, 2018)")
bottomPad.cd()
bottomPad.SetTopMargin(0)
bottomPad.SetBottomMargin(0.3)
pullHist.SetTitle()
pullHist.SetMarkerStyle(24)
pullHist.GetYaxis().SetTitle("Pull = #frac{MC - Fit}{#sigma_{MC}}")
pullHist.GetYaxis().SetTitleSize(0.1)
pullHist.GetYaxis().SetLabelSize(0.13)
pullHist.GetYaxis().SetNdivisions(2)
pullHist.GetYaxis().SetLimits(-3.9, 3.9)
pullHist.GetYaxis().SetTickLength(0.01)
pullHist.GetYaxis().SetTitleOffset(0.4)
pullHist.GetXaxis().SetTitle("M_{Z} [GeV]")
pullHist.GetXaxis().SetTitleSize(0.13)
pullHist.GetXaxis().SetLabelSize(0.13)
pullHist.GetXaxis().SetTitleOffset(1.1)
pullHist.GetXaxis().SetLimits(74.5, 105.5)
pullHist.Draw("ap")
bottomPad.Update()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cComp
